# Week-2 The Battle of Neighnorhoods

In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
import os 
import numpy as np
import pandas as pd
import datetime as dt 
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
print('Libraries imported.')

Libraries imported.


In [3]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [4]:
df_ppd.head(5)

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [5]:
df_ppd.shape

(1029749, 16)

In [6]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")
df_affordable.head()

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
855,BARONSMEAD ROAD,2375000.0


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from sklearn.cluster import KMeans

In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [13]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df_affordable.head()

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2400000.0,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"


In [16]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:
df_affordable.head()

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457


In [18]:
df = df_affordable.drop(columns=['city_coord'])
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [19]:
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [20]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
map_london

In [21]:
CLIENT_ID = 'EQCETFTKTGPVRSP3UUKNU25110D3GD3VBLIZYE3WHSYVZVSE' 
CLIENT_SECRET = 'CFWNP2O3RNMIYAVEJBN0UFACWAGPKNN04EQCZ32JQV1Z4P2B' 
VERSION = '20181206' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EQCETFTKTGPVRSP3UUKNU25110D3GD3VBLIZYE3WHSYVZVSE
CLIENT_SECRET:CFWNP2O3RNMIYAVEJBN0UFACWAGPKNN04EQCZ32JQV1Z4P2B


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [23]:
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [24]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
7,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
8,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
9,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant


In [25]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,29,29,29,29,29,29
ANHALT ROAD,18,18,18,18,18,18
ANSDELL TERRACE,42,42,42,42,42,42
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,3,3,3,3,3,3
BELVEDERE DRIVE,2,2,2,2,2,2
BICKENHALL STREET,62,62,62,62,62,62
BIRCHLANDS AVENUE,11,11,11,11,11,11


In [26]:
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))
location_venues.shape

There are 341 uniques categories.


(4300, 7)

In [27]:
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['Street'] = location_venues['Street'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.034483,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
1,ANHALT ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.055556,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
2,ANSDELL TERRACE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.023810,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
3,APPLEGARTH ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
4,BARONSMEAD ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
5,BEAUCLERC ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
6,BELVEDERE DRIVE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
7,BICKENHALL STREET,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.016129,0.000000,0.016129,0.016129,0.0
8,BIRCHLANDS AVENUE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
9,BRAMPTON GROVE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0


In [29]:
london_grouped.shape

(150, 342)

In [30]:
num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1        Coffee Shop  0.10
2  Indian Restaurant  0.07
3                Bar  0.07
4         Restaurant  0.07


----ANHALT ROAD----
               venue  freq
0                Pub  0.22
1      Grocery Store  0.11
2              Diner  0.06
3             Garden  0.06
4  Outdoor Sculpture  0.06


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1               Hotel  0.07
2      Clothing Store  0.07
3          Restaurant  0.07
4           Juice Bar  0.07


----APPLEGARTH ROAD----
            venue  freq
0             Pub   0.2
1       Nightclub   0.2
2          Casino   0.2
3  Sandwich Place   0.2
4             Bar   0.2


----BARONSMEAD ROAD----
              venue  freq
0   Nature Preserve  0.08
1   Thai Restaurant  0.08
2  Community Center  0.08
3        Restaurant  0.08
4       Pizza Place  0.08


----BEAUCLERC ROAD----
               venue  freq
0    Harbor / Marina  0.33
1    Au

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [33]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']
for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Coffee Shop,Indian Restaurant,Restaurant,Pub,Bar,Brewery,Breakfast Spot,Museum,Fish & Chips Shop
1,ANHALT ROAD,Pub,Grocery Store,Plaza,Cocktail Bar,French Restaurant,Garden,Outdoor Sculpture,English Restaurant,Park,Gym / Fitness Center
2,ANSDELL TERRACE,Restaurant,Hotel,Juice Bar,Clothing Store,Italian Restaurant,Indian Restaurant,Pub,Bakery,English Restaurant,Sporting Goods Shop
3,APPLEGARTH ROAD,Bar,Sandwich Place,Pub,Casino,Nightclub,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
4,BARONSMEAD ROAD,Movie Theater,Food & Drink Shop,Park,Breakfast Spot,Farmers Market,Restaurant,Coffee Shop,Nature Preserve,Thai Restaurant,Community Center


In [34]:
venues_sorted.shape

(150, 11)

In [35]:
london_grouped.shape

(150, 342)

In [36]:
london_grouped=df

In [37]:
kclusters = 5
london_grouped_clustering = london_grouped.drop('Street', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)
kmeans.labels_[0:50]

array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [38]:
london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [39]:
london_grouped_clustering.shape

(162, 4)

In [40]:
df.shape

(162, 4)

In [41]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [42]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [43]:
london_grouped_clustering['Cluster Labels'] = kmeans.labels_
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')
london_grouped_clustering.head(30) 

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Coffee Shop,Indian Restaurant,Restaurant,Pub,Bar,Brewery,Breakfast Spot,Museum,Fish & Chips Shop
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,Grocery Store,Plaza,Cocktail Bar,French Restaurant,Garden,Outdoor Sculpture,English Restaurant,Park,Gym / Fitness Center
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Restaurant,Hotel,Juice Bar,Clothing Store,Italian Restaurant,Indian Restaurant,Pub,Bakery,English Restaurant,Sporting Goods Shop
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Bar,Sandwich Place,Pub,Casino,Nightclub,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Movie Theater,Food & Drink Shop,Park,Breakfast Spot,Farmers Market,Restaurant,Coffee Shop,Nature Preserve,Thai Restaurant,Community Center
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Automotive Shop,Pizza Place,Harbor / Marina,Zoo,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796,2,Pool,Playground,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Pizza Place,Restaurant,Café,Italian Restaurant,Gastropub,Bar,Movie Theater,Chinese Restaurant,Garden,Bakery
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,French Restaurant,Pub,Bakery,Coffee Shop,Breakfast Spot,Chinese Restaurant,Lake,Train Station,Brewery,Department Store
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Home Service,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant


In [44]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Restaurant,Hotel,Juice Bar,Clothing Store,Italian Restaurant,Indian Restaurant,Pub,Bakery,English Restaurant,Sporting Goods Shop
1215,2208500.0,Pizza Place,Restaurant,Café,Italian Restaurant,Gastropub,Bar,Movie Theater,Chinese Restaurant,Garden,Bakery
1253,2217000.0,French Restaurant,Pub,Bakery,Coffee Shop,Breakfast Spot,Chinese Restaurant,Lake,Train Station,Brewery,Department Store
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,2250000.0,Hotel,Coffee Shop,Seafood Restaurant,Outdoor Sculpture,Bakery,Grocery Store,Supermarket,Pharmacy,Restaurant,Outdoor Supply Store


In [46]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Coffee Shop,Indian Restaurant,Restaurant,Pub,Bar,Brewery,Breakfast Spot,Museum,Fish & Chips Shop
981,2480000.0,Automotive Shop,Pizza Place,Harbor / Marina,Zoo,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1553,2456875.0,Home Service,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1980,2492500.0,Supermarket,English Restaurant,Park,Café,Hardware Store,Coffee Shop,Gym,Rental Car Location,Fast Food Restaurant,Pub
2136,2461000.0,Spa,Sports Club,Windmill,Soccer Field,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit


In [47]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Movie Theater,Food & Drink Shop,Park,Breakfast Spot,Farmers Market,Restaurant,Coffee Shop,Nature Preserve,Thai Restaurant,Community Center
1102,2340000.0,Pool,Playground,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
2068,2375000.0,Pub,Park,Hotel,Grocery Store,Pizza Place,Indian Restaurant,Yoga Studio,Gastropub,Thai Restaurant,Tennis Court
2129,2379652.7,Pub,Grocery Store,Indian Restaurant,Coffee Shop,Yoga Studio,Park,Bakery,Hotel,Pizza Place,Record Shop
2944,2367500.0,Hotel,Pub,Garden,Café,Italian Restaurant,Coffee Shop,Bar,Mediterranean Restaurant,Chinese Restaurant,Gym / Fitness Center


In [48]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,Plaza,Cocktail Bar,French Restaurant,Garden,Outdoor Sculpture,English Restaurant,Park,Gym / Fitness Center
422,2400000.0,Bar,Sandwich Place,Pub,Casino,Nightclub,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1632,2397132.0,Health & Beauty Service,Grocery Store,Gym / Fitness Center,Italian Restaurant,Coffee Shop,Zoo,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1797,2400000.0,Park,Zoo,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1914,2445000.0,Gym,Dance Studio,Athletics & Sports,Bar,Grocery Store,Fishing Store,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant


In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Farm,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2406,2.286679e+06,Café,Italian Restaurant,Bar,Pub,Coffee Shop,French Restaurant,Park,Beer Store,Greek Restaurant,Gourmet Shop
2686,2.287500e+06,Pub,Brewery,Gym / Fitness Center,Gift Shop,Art Museum,Zoo,Farm,English Restaurant,Ethiopian Restaurant,Event Space
3377,2.298000e+06,Hotel,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
4285,2.265000e+06,Pub,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
